<a href="https://colab.research.google.com/github/GEETHIKACHINNI2/chinni-27/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?


Yes, **averaging the validation accuracy across multiple splits** (such as in **cross-validation**) typically **gives more consistent and reliable results** compared to relying on a single validation split. This is because it reduces the impact of any single data partition and provides a more comprehensive and stable measure of your model's performance.

### Why Does Averaging Across Multiple Splits Work?

1. **Reduces Variance**:
   - If you use just one **training/validation split**, your model's performance can be influenced by the specific data points in that split. Some splits might contain more challenging examples, while others might be easier, leading to results that are not necessarily representative of your model's true performance on unseen data.
   - **Multiple splits** smooth out these fluctuations because the model is evaluated on a variety of data subsets, which provides a more **robust and generalized estimate** of how well it is likely to perform in real-world scenarios.

2. **Better Estimation of Performance**:
   - With **cross-validation** (e.g., **k-fold cross-validation**), you evaluate your model multiple times, each time on a different subset of the data. This allows each data point to be used both for training and testing, reducing the risk that an individual split might be unrepresentative or biased.
   - For example, if your dataset contains noisy or difficult samples, a single validation set might give misleading results. Averaging over multiple folds ensures that these outliers or challenging samples don’t overly influence the model evaluation.

3. **Helps Detect Overfitting**:
   - If your model performs well on some splits but poorly on others, this could indicate that it is **overfitting** to specific parts of the data. Cross-validation helps identify this by providing insights into how consistently the model generalizes across different subsets of the data. If there's large variability in the accuracy scores across splits, you may need to adjust your model or preprocessing to improve generalization.

4. **More Reliable Metric**:
   - The **average validation accuracy** across multiple splits gives you a better understanding of your model’s **true generalization performance**. Instead of having one potentially misleading score (based on a random split), you get a more **stable and trustworthy estimate** of the model’s capabilities.

---

### Example

Let’s say you’re using a **5-fold cross-validation** approach, which means your dataset is split into 5 equal parts, and the model is trained and tested 5 times, each time on a different subset. You then average the performance across these 5 splits.

- **Fold 1**: Accuracy = 85%
- **Fold 2**: Accuracy = 90%
- **Fold 3**: Accuracy = 88%
- **Fold 4**: Accuracy = 87%
- **Fold 5**: Accuracy = 86%

The **average accuracy** would be:

\[
\text{Average Accuracy} = \frac{85\% + 90\% + 88\% + 87\% + 86\%}{5} = 87.2\%
\]

By comparing this **average accuracy** with the performance from a single split (which could vary significantly), you get a more reliable and stable measure of how your model is likely to perform on unseen data.

---

### Key Benefits of Averaging Validation Accuracy Across Multiple Splits:

1. **Minimizes the Risk of Overfitting**: Averaging helps ensure that the model isn't just memorizing patterns in a specific split of the data.
   
2. **Better Generalization**: The model is evaluated on different data subsets, helping you get a more accurate understanding of how it generalizes to unseen data.

3. **Smoother Results**: Averaging across multiple splits provides a more **smooth and reliable estimate** of the model’s performance, reducing the noise that can come from a single, potentially unrepresentative test set.

4. **Identifying Potential Issues**: If your model shows large variation across splits (e.g., some folds with much lower accuracy), it can help highlight issues with the model, data, or splits that might need to be addressed.

---

### Conclusion:

Averaging validation accuracy across multiple splits indeed **leads to more consistent and stable results**. This technique is widely used because it provides a **better, more robust estimate** of model performance, reduces the likelihood of overfitting to a specific dataset split, and helps detect issues that might not be apparent with a single validation set. It's one of the best ways to assess how well your model is likely to perform when deployed on unseen, real-world data.



2. Does it give more accurate estimate of test accuracy?

Averaging the validation accuracy across multiple splits in cross-validation **does provide a more accurate estimate of test accuracy**, but it’s important to clarify what we mean by **test accuracy** and how it relates to cross-validation.

### Key Points:

1. **Validation Accuracy vs. Test Accuracy**:
   - **Validation accuracy** refers to the accuracy of the model on the validation sets during cross-validation. These validation sets are subsets of your training data used for model evaluation during training, but they are not the test set.
   - **Test accuracy** refers to the accuracy of the model on a completely unseen, held-out test set (which has never been used during training or validation). This is the **final evaluation** of the model after all training and hyperparameter tuning are completed.

2. **Cross-Validation as a Proxy for Test Accuracy**:
   - **Cross-validation** (e.g., k-fold cross-validation) provides an estimate of how well your model will generalize to new, unseen data. By averaging the validation accuracy across multiple splits, you are obtaining a more stable and reliable estimate of how your model will perform in practice.
   - While **cross-validation accuracy** and **test accuracy** are not identical (because cross-validation is still done on the training data, not the test set), cross-validation is generally the **best available estimate** of your model's likely performance on a held-out test set, especially if the dataset is small or you don't have a separate test set.

### Why Does Cross-Validation Provide a Good Estimate of Test Accuracy?

1. **More Reliable Estimate of Generalization**:
   - Cross-validation tests the model on different subsets of the data (folds), meaning that each data point gets the chance to be in the validation set, and each model is trained on different training sets. This provides a **more generalizable estimate** of performance compared to a single validation split.
   - Since the cross-validation splits are created randomly, it’s likely that each model has seen **a variety of data points**, allowing it to learn a broader representation of the data, which in turn improves the estimate of its real-world performance.

2. **Mitigates Overfitting**:
   - A model that performs well on a single validation set might be **overfitting** to that specific subset of data. By averaging across multiple validation splits, you reduce the risk of **overfitting to a single validation set**, which could give an overly optimistic view of the model’s test accuracy.
   - In contrast, if you rely on a single validation set (or holdout test set), the performance estimate might be subject to randomness or biases from that particular split. Cross-validation helps to smooth out such variability.

3. **Leverages All Data**:
   - Cross-validation ensures that every data point is used for both training and validation, which is especially useful when you have **limited data**. This maximizes the use of available data for both training and validation, leading to a more robust model.
   - For small datasets, this is particularly important because every data point is used in validation at least once, providing a more accurate representation of the model's generalization performance.

---

### Example of Why Cross-Validation Provides a Good Estimate of Test Accuracy

Imagine you have a dataset with 1000 data points, and you split the data into 5 folds for **5-fold cross-validation**. You train your model on different combinations of 800 points and test it on the remaining 200 points for each fold. If you repeat this process several times and average the validation accuracies, you obtain a more stable estimate of how well the model will perform on unseen data.

Now, let's say you have a **separate test set** of 200 samples that was not used during training or validation. Once you’ve completed cross-validation, you could then evaluate your model on the **test set**. Typically, the performance on this test set will be somewhat close to the **average cross-validation performance**, assuming no data leakage or major issues with the model.

In some cases, test accuracy may be slightly lower than the cross-validation accuracy, because the test set represents new, completely unseen data, while cross-validation uses data that the model has already trained on (even though it’s not the same exact data each time).

---

### Summary: Does Cross-Validation Provide a More Accurate Estimate of Test Accuracy?

Yes, **cross-validation provides a more accurate estimate of test accuracy** than relying on a single train-test split. This is because:

1. **Multiple splits** provide a more stable estimate of how the model generalizes to unseen data.
2. **Averaging results** from multiple folds helps mitigate variability caused by random data partitioning.
3. It **reduces the risk of overfitting** to a specific validation split.
4. It allows you to **maximize the use of your data**, which is especially important for small datasets.

However, it’s important to note that **cross-validation** is still an approximation of the true test accuracy, as it does not directly use the held-out test set (which is only used after final model selection). Cross-validation gives a robust estimate of the model’s expected performance, but **test accuracy** is still the ultimate measure of how well the model generalizes to truly unseen data.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?


Yes, **cross-validation** helps **reduce the risk of overfitting** to a specific validation split by evaluating the model on multiple different subsets of the data, rather than just a single validation set. Here's how it works in more detail:

### 1. **What is Overfitting to a Validation Split?**

Overfitting occurs when a model learns patterns from the training data that do not generalize well to new, unseen data. In the case of a **validation split**, overfitting happens if the model becomes overly tailored to the specific data points in that validation set, leading to high performance on that split but poor generalization to unseen data.

For example:
- If a model is trained on one specific **training set** and then evaluated on a **validation set** (for a single split), it may perform very well if the validation set contains examples that are very similar to the training set.
- However, if the validation set contains very different examples or some challenging samples, the model may perform poorly, and the evaluation on that split might give a misleadingly **high or low** performance estimate.

### 2. **How Cross-Validation Reduces Overfitting to a Single Split**

Cross-validation, especially **k-fold cross-validation**, helps address this issue by:

- **Training and validating on different splits**: In k-fold cross-validation, the data is split into **k** different folds (subsets). The model is trained and tested **k times**, each time on a different subset of the data. Each fold serves as the **validation set** once, and the remaining k-1 folds are used for training.
  
- **Each data point gets to be in the validation set at least once**: Because the model is trained on different data each time (since the training set changes with each fold), it is less likely to become overly tailored to any one particular split of data. Every data point is used for both training and validation, which allows the model to learn patterns that are more **generalizable** to unseen data.

- **Reduces dependence on a single validation set**: By using multiple different validation sets (i.e., each fold), cross-validation averages out the potential biases or random quirks that might appear in a single validation set. If a particular split happens to have easier or harder examples, it will only affect the overall performance estimate a little, rather than significantly skewing the results.

---

### 3. **Illustrative Example**

Suppose you have a dataset with 100 samples, and you're doing **5-fold cross-validation**. You would split the data into 5 parts (folds):

- **Fold 1**: Training on 80 samples, testing on 20 samples.
- **Fold 2**: Training on 80 samples, testing on 20 samples.
- **Fold 3**: Training on 80 samples, testing on 20 samples.
- **Fold 4**: Training on 80 samples, testing on 20 samples.
- **Fold 5**: Training on 80 samples, testing on 20 samples.

In each fold, the model is trained on 80% of the data and tested on 20% of the data. After completing all 5 folds, you have 5 different validation results. The final performance metric is averaged across all folds.

#### Without Cross-Validation (Single Split):
- Suppose you perform a simple **train/test split** where 80% of the data is used for training, and 20% is used for validation. The model might perform well or poorly based on that specific split.
  - If the validation set happens to have relatively easy examples, the model might appear to perform better than it actually would on unseen data.
  - If the validation set is difficult or doesn't represent the full range of the data, the model might appear to perform poorly.

#### With Cross-Validation (Multiple Splits):
- In cross-validation, the model is trained on different subsets of data and tested on different validation sets multiple times. This reduces the likelihood that the model will be overfitted to the quirks or specificities of a single training/validation split.
- Since the model is tested across **multiple splits**, it’s less likely to become overly specialized to one particular fold. The final performance estimate is averaged across all folds, leading to a more reliable and generalizable result.

### 4. **How Cross-Validation Helps in Practice**

1. **Less Bias and Variance in Model Evaluation**:
   - **Overfitting to a validation set** can lead to performance estimates that are too optimistic or pessimistic, depending on how representative that validation set is. Cross-validation helps reduce this **bias** and also the **variance** caused by random splits of the data.

2. **Better Generalization**:
   - By ensuring that the model is trained and evaluated on different data subsets, cross-validation encourages the model to **learn general patterns** rather than specific patterns from a particular subset of the data, leading to better generalization.

3. **More Reliable Hyperparameter Tuning**:
   - When you're tuning model hyperparameters (e.g., learning rate, regularization), cross-validation helps ensure that the hyperparameter choices are **more stable and generalizable**. Without cross-validation, hyperparameter tuning might be overly reliant on how the model performs on a single validation set.

### 5. **Practical Example: Overfitting with a Single Validation Split**

Imagine you have a small dataset with 100 samples, and you decide to use an **80-20 train-test split**. If the 20 samples in the test set happen to be easier or more similar to the training set, the model might achieve a high accuracy on the test set. But if those 20 samples were harder or more different, the accuracy could drop significantly, and this might give you an overly optimistic or pessimistic evaluation of your model.

Now, using **k-fold cross-validation** (e.g., 5-fold), the model is trained and tested on different subsets. By averaging the performance across all folds, the overall evaluation becomes **more stable**, and the risk of overfitting to one specific split is minimized.

---

### Conclusion

In summary, **averaging validation accuracy across multiple splits** (through cross-validation) reduces the risk of overfitting to any specific validation split. By training and testing the model on different subsets of the data multiple times, cross-validation ensures that the model generalizes better and that the performance estimate is less dependent on the random quirks of a single validation set. This is why cross-validation is considered a more reliable way to estimate model performance compared to a single train/test split.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

When dealing with a **very small training or validation dataset**, **increasing the number of iterations** (such as in **cross-validation**) can help improve the robustness of your model evaluation, but it **does not directly address the core issues** that arise from having limited data. Let's break down how increasing iterations can help and what additional strategies might be needed.

### 1. **Increasing Iterations in Cross-Validation**

If you increase the number of iterations (i.e., the number of folds in **k-fold cross-validation**), it can help in a few ways:

- **More splits (more folds)**: In **k-fold cross-validation**, each fold acts as a validation set, and the model is trained on the remaining data. If you increase the number of folds, for example, using **10-fold cross-validation** instead of **5-fold**, you increase the number of iterations (or validations) the model undergoes, which can provide a more stable estimate of model performance. This helps reduce the bias introduced by any single split.
  
  - **Example**: In 5-fold cross-validation, the data is split into 5 subsets, and each subset is used for testing once. If you increase this to 10-fold, the model will be tested more times, which means each fold will be validated on different data, helping reduce the risk of overfitting to a particular split.

  - **More data for training**: Increasing the number of folds means each time the model is trained, it has access to more of the data. This is especially helpful when you have a small dataset because each training iteration is performed on a slightly different subset of data, ensuring that the model learns from as many data points as possible.

However, while increasing the number of folds can give you a more consistent evaluation, it **does not fundamentally address the problem of having limited data**, and there are a few limitations:

- **Increased computational cost**: More folds mean more model training and testing. This is fine if computational resources are available but can become expensive for large models or very small datasets where training might be time-consuming.
  
- **Potential for overfitting with very small datasets**: Even with multiple iterations, if the dataset is too small, there may still be a risk of **overfitting** due to insufficient data to capture the full range of patterns in the underlying distribution. The model may still perform well in validation but fail to generalize to real-world data that might differ slightly from the small training/validation sets.

### 2. **Additional Strategies for Small Datasets**

If your training or validation dataset is too small, there are other strategies you can consider to improve your model’s generalization ability beyond just increasing the number of iterations:

#### a. **Data Augmentation (for specific data types like images or text)**
   - **Image Data**: For tasks like image classification, you can apply techniques like **rotation, flipping, scaling, cropping**, etc., to artificially increase the size of your training set.
   - **Text Data**: For NLP tasks, you can use techniques like **synonym replacement, back-translation, or sentence paraphrasing** to generate more examples from the available data.
   
   Data augmentation helps create variations in the data, increasing the diversity of the training set, which is particularly useful in small datasets.

#### b. **Transfer Learning**
   - For tasks such as image classification or natural language processing, you can use **pre-trained models** (e.g., from large datasets like ImageNet for images or BERT for text) and fine-tune them on your small dataset. This is known as **transfer learning**.
   - Since the model has already learned many useful features from a larger dataset, it can apply that knowledge to your small dataset, which can significantly improve performance even with limited data.

#### c. **Bootstrapping (Resampling)**
   - **Bootstrapping** involves repeatedly sampling from the dataset with replacement to create multiple new training sets. These can then be used to train the model multiple times and average the results.
   - This technique can increase the effective size of your training set and provide more stable performance estimates, although it still may not fully compensate for very small datasets.

#### d. **Synthetic Data Generation**
   - For certain types of problems, you can use techniques like **SMOTE** (Synthetic Minority Over-sampling Technique) to generate synthetic data points, particularly in classification tasks with imbalanced classes.
   - For example, if your dataset has few samples in one class, SMOTE can create synthetic instances by interpolating between the minority class points.

#### e. **Regularization**
   - **Regularization** methods like **L1 (Lasso)**, **L2 (Ridge)**, and **Dropout** (for neural networks) can help prevent overfitting, especially in situations where data is limited. These methods penalize overly complex models and encourage simpler, more generalizable solutions.
   
   - In the case of a small dataset, regularization can reduce the risk that the model will memorize the training data rather than learning general patterns.

#### f. **Simplify the Model**
   - If you have very limited data, using a simpler model (with fewer parameters) might help. A complex model with many parameters (like deep neural networks) is more likely to overfit when trained on small datasets.
   - For instance, using a **linear model** or **decision tree** might provide better generalization compared to a deep neural network when data is scarce.

#### g. **Use of Domain Knowledge**
   - If you have domain expertise in the problem, you can often improve generalization by **engineering more informative features** or using prior knowledge to constrain the model. This might involve transforming the data, applying known relations, or even using a hybrid model that combines expert rules with machine learning.

---

### 3. **Conclusion**

While **increasing the number of iterations (e.g., more folds in cross-validation)** can help mitigate the risk of overfitting to a specific validation set and provide a more consistent performance estimate, it **does not directly solve the problem of having a very small dataset**. The key issue with small datasets is that there simply may not be enough data to capture the full range of patterns in the underlying distribution, making it difficult for models to generalize well.

To effectively deal with a very small training or validation dataset, you should consider combining **multiple strategies**:

1. **Use cross-validation with more folds** to improve evaluation consistency.
2. Apply **data augmentation** or **synthetic data generation** if applicable.
3. Consider **transfer learning** if you can leverage pre-trained models.
4. Use **regularization** techniques to prevent overfitting.
5. Consider **simpler models** that are less likely to overfit on limited data.

In many cases, the combination of these techniques will give you a more robust model despite the challenges of a small dataset.


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.